### Problema de negócio:    
      Aneel - órgão regulador. Sabemos o consumo por estado, não conhecemos o perfil do cliente. Preciso agrupar o perfil cliente de acordo com o nível de consumo independe de onde eles moram, pois, quero fazer uma atividade direcionada.

      Precisamos construir uma Máquina Preditiva (MP), a partir do consumo de energia de cliente. Agrupe os consumidores por similaridades, afim de compreender o comportamento dos clientes e sua relação com o consumo de energia.